In [12]:
#spark.stop()

# News categorization

## Logistic regression

In [126]:
from pyspark import SparkConf, SparkContext,SparkSession
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col, trim, regexp_replace,split,expr,lit

ImportError: cannot import name 'SparkSession' from 'pyspark' (/home/gbetoho.adede/.local/lib/python3.8/site-packages/pyspark/__init__.py)

In [61]:
spark = (SparkSession.builder 
    .master("spark://node02:7077") 
    .appName("NewsCategorisation") 
    #.config('spark.driver.cores','4')
    #.config("spark.executorEnv.LD_LIBRARY_PATH", "/home/team1337/.local/easybuild_new/software/Python/3.8.6-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/libffi/3.3-GCCcore-10.2.0/lib64:/home/team1337/.local/easybuild_new/software/GMP/6.2.0-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/XZ/5.2.5-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/SQLite/3.33.0-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/Tcl/8.6.10-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/libreadline/8.0-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/ncurses/6.2-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/bzip2/1.0.8-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/binutils/2.35-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/zlib/1.2.11-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/GCCcore/10.2.0/lib64") \
    #.config("spark.pyspark.python", "/home/team1337/.local/easybuild_new/software/Python/3.8.6-GCCcore-10.2.0/bin/python3") \
    .getOrCreate()
        )
spark

In [62]:
# Load data
#df = spark.read.csv("input/news.csv", header=True, inferSchema=True)
df = spark.read.parquet("input/news.parquet", header=True, inferSchema=True)

In [63]:
df.rdd.getNumPartitions()

9

In [66]:
num_partitions=5*40
df= df.repartition(num_partitions).cache()

In [67]:
df.rdd.getNumPartitions()

200

In [68]:
df=df.cache()

24/06/04 19:20:53 WARN CacheManager: Asked to cache already cached data.


In [71]:
df.count()

1716608

In [72]:
df.show()

+--------------+--------------------+
|category_label|description_filtered|
+--------------+--------------------+
|          10.0|every bozo need k...|
|          11.0|best redness past...|
|          10.0|fashion show used...|
|           9.0|challenge present...|
|          10.0|man vintage show ...|
|          11.0|hidden mickey spo...|
|          10.0|next fashion uppe...|
|          11.0|work home rabbi d...|
|           9.0|mommy manage nt g...|
|           9.0|ontogenesis hormo...|
|          10.0|7 gross grooming ...|
|           9.0|period display bo...|
|          10.0|transformation ph...|
|          10.0|morena baccarin g...|
|          11.0|expat recovery ro...|
|          11.0|amsterdam diverse...|
|          11.0|view afar make ma...|
|          10.0|new house york fa...|
|           9.0|5 dumbest affair ...|
|           9.0|   valentine day kid|
+--------------+--------------------+
only showing top 20 rows



In [73]:
from pyspark.sql.functions import col, desc,when,split,format_number,explode

In [74]:
from pyspark.sql.functions import split

# Create a new DataFrame with description_filtered as arrays
df= df.withColumn('description_filtered', split(col('description_filtered'), ' '))
# Show the new DataFrame
df.show(truncate=False)


+--------------+----------------------------------------------------------------------------------------------------------------+
|category_label|description_filtered                                                                                            |
+--------------+----------------------------------------------------------------------------------------------------------------+
|10.0          |[every, bozo, need, know, fashion]                                                                              |
|11.0          |[best, redness, pasta, sauce, italian, love, life, ve, never, heard]                                            |
|10.0          |[fashion, show, used, represent, raucous, liaison, video]                                                       |
|9.0           |[challenge, present, girl, constitute, dying]                                                                   |
|10.0          |[man, vintage, show, delivers, carefully, curated, habiliment, culled, aro

In [75]:
exploded_df=df.select(explode(df.description_filtered)).alias('words')

In [76]:
#df=df.unpersist()

In [77]:
unique_words=exploded_df.distinct()

In [78]:
unique_words=unique_words.cache()

In [79]:
unique_words.show()

+--------+
|     col|
+--------+
|     art|
|   oscar|
|newlywed|
|  harder|
|  teigen|
|   still|
|   monte|
|   carlo|
| hitched|
|  filing|
|    pant|
|  travel|
|  bazaar|
|  poetry|
|  outfit|
|   1970s|
| elevate|
|  online|
| embrace|
|  patton|
+--------+
only showing top 20 rows



In [80]:
vocabulary_size=unique_words.count()
vocabulary_size

128622

In [81]:
unique_words=unique_words.unpersist()

In [82]:
from math import ceil,log2

In [83]:
n=ceil(log2(vocabulary_size))
n

17

In [84]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier,MultilayerPerceptronClassifier,NaiveBayes,LinearSVC,DecisionTreeClassifier,OneVsRest

In [95]:
# Function to evaluate model and get best parameters
def evaluate_model(model, model_name):
    print('Training the model')

    # Train the model using cross-validation
    fitted_model = model.fit(train_set)

    # Get the best model from cross-validation
    best_model = fitted_model.stages[-1].bestModel

    print('Making predictions on the training set')
    # Make predictions on the training set
    train_predictions = fitted_model.transform(train_set)

    print('Making predictions on the test set')
    # Make predictions on the test set
    test_predictions = fitted_model.transform(test_set)

    # Initialize the evaluator
    evaluator = MulticlassClassificationEvaluator(labelCol="category_label", predictionCol="prediction", metricName="accuracy")

    print('Evaluating the model on training set')
    # Evaluate the model on the training set
    train_accuracy = evaluator.evaluate(train_predictions)

    print('Evaluating the model on test set')
    # Evaluate the model on the test set
    test_accuracy = evaluator.evaluate(test_predictions)

    print(f"{model_name} Train Accuracy: {train_accuracy}")
    print(f"{model_name} Test Accuracy: {test_accuracy}")

    # Print the best parameters
    print(f"Best parameters for {model_name}:")
    
    for param, value in best_model.extractParamMap().items():
        print(f"  {param.name}: {value}")

    return train_accuracy, test_accuracy,best_model

In [89]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, NaiveBayes,DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import numpy as np

In [97]:




# Define the HashingTF and IDF stages
hashingTF = HashingTF(inputCol="description_filtered", outputCol="rawFeatures", numFeatures=2**n)
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Define the classifiers
lr = LogisticRegression(labelCol="category_label", featuresCol="features")

nb = NaiveBayes(labelCol="category_label", featuresCol="features")

# Define parameter grids
paramGrid_nb = ParamGridBuilder() \
    .addGrid(nb.smoothing, [0.5, 1.0, 2.0]) \
    .build()



reg_values = np.logspace(-4, 4, num=100)

l1_ratios = np.linspace(0, 1, num=10) 

paramGrid_lr = (ParamGridBuilder() 
    .addGrid(lr.regParam, reg_values) 
    #.addGrid(lr.elasticNetParam, l1_ratios) 
    .build())



paramGrid_dt = (ParamGridBuilder() 
    #.addGrid(dt.numTrees, [10, 50, 100]) 
    .addGrid(dt.maxDepth, [5, 10, 20]) 
    .build())

# Create cross validators
cv_nb = CrossValidator(estimator=nb, estimatorParamMaps=ParamGridBuilder().build(), #paramGrid_nb,
                       evaluator=MulticlassClassificationEvaluator(labelCol="category_label", predictionCol="prediction", metricName="accuracy"),
                       numFolds=3, parallelism=1)

cv_lr = CrossValidator(estimator=lr, estimatorParamMaps=ParamGridBuilder().build(), #paramGrid_lr,
                       evaluator=MulticlassClassificationEvaluator(labelCol="category_label", predictionCol="prediction", metricName="accuracy"),
                       numFolds=3, parallelism=1)

cv_dt = CrossValidator(estimator=dt, estimatorParamMaps=ParamGridBuilder().build(), # paramGrid_rf,
                       evaluator=MulticlassClassificationEvaluator(labelCol="category_label", predictionCol="prediction", metricName="accuracy"),
                       numFolds=3,parallelism=3)

# Create pipelines
pipeline_nb = Pipeline(stages=[hashingTF, idf, cv_nb])
pipeline_lr = Pipeline(stages=[hashingTF, idf, cv_lr])

# Split data
(train_set, test_set) = df.randomSplit([0.8, 0.2], seed=0)



# Evaluate models using the evaluate_model function
results = {}
for model_pipeline, model_name in [
    (pipeline_nb, "Naive Bayes"),
    #(pipeline_lr, "Logistic Regression"),
    
]:
    print(f'Evaluating {model_name}')
    train_accuracy, test_accuracy,best_model = evaluate_model(model_pipeline, model_name)
    results[model_name] = {"train_accuracy": train_accuracy, "test_accuracy": test_accuracy}
    
    

Evaluating Naive Bayes
Training the model


24/06/04 19:56:22 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:37 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:39 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:44 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:45 WARN DAGScheduler: Broadcasting large task binary with size 34.1 MiB
24/06/04 19:56:47 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:48 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:51 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:55 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:57 WARN DAGScheduler: Broadcasting large task binary with size 34.1 MiB
24/06/04 19:56:58 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:59 WARN DAGScheduler: Broadcasting la

Making predictions on the training set
Making predictions on the test set
Evaluating the model on training set


24/06/04 19:57:17 WARN DAGScheduler: Broadcasting large task binary with size 34.1 MiB


Evaluating the model on test set


24/06/04 19:57:23 WARN DAGScheduler: Broadcasting large task binary with size 34.1 MiB


Naive Bayes Train Accuracy: 0.786215323314107
Naive Bayes Test Accuracy: 0.7531130303074436
Best parameters for Naive Bayes:
  featuresCol: features
  labelCol: category_label
  modelType: multinomial
  predictionCol: prediction
  probabilityCol: probability
  rawPredictionCol: rawPrediction
  smoothing: 1.0


In [102]:
best_model.save('pp')

24/06/04 20:03:43 WARN TaskSetManager: Stage 216 contains a task of very large size (33450 KiB). The maximum recommended task size is 1000 KiB.


In [117]:
from pyspark.ml.classification import NaiveBayesModel
test_set=te
loaded_model = NaiveBayesModel.load('pp')

24/06/04 20:31:50 ERROR Instrumentation: java.lang.IllegalArgumentException: requirement failed: Error loading metadata: Expected class name org.apache.spark.ml.PipelineModel but found class name org.apache.spark.ml.classification.NaiveBayesModel
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.ml.util.DefaultParamsReader$.parseMetadata(ReadWrite.scala:610)
	at org.apache.spark.ml.util.DefaultParamsReader$.loadMetadata(ReadWrite.scala:588)
	at org.apache.spark.ml.Pipeline$SharedReadWrite$.$anonfun$load$3(Pipeline.scala:269)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:191)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:191)
	at org.apache.spark.ml.Pipeline$SharedReadWrite$.load(Pipeline.scala:268)
	at org.apache.spark.ml.PipelineModel$PipelineModelReader.$anonfun$load$7(Pipeline.scala:356)
	at org.apache.spark.ml.MLEvents.withLoadInstanceEvent(events

IllegalArgumentException: requirement failed: Error loading metadata: Expected class name org.apache.spark.ml.PipelineModel but found class name org.apache.spark.ml.classification.NaiveBayesModel

In [ ]:
n = 17  # Assuming n is defined somewhere
hashingTF = HashingTF(inputCol="description_filtered", outputCol="rawFeatures", numFeatures=2**n)
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Apply HashingTF to the test set
test_set_with_raw_features = hashingTF.transform(test_set)

# Fit IDF on the training set to get the IDF model
idf_model = idf.fit(test_set_with_raw_features)

# Apply IDF to the test set
test_set_with_features = idf_model.transform(test_set_with_raw_features)

# Show the test set with the new features column
test_set_with_features.select("description_filtered", "rawFeatures", "features").show(5, truncate=False)

In [120]:
loaded_model=NaiveBayesModel.load('pp')
# Verify the model is loaded by checking its parameters or making predictions
print("Loaded model parameters:")
for param, value in loaded_model.extractParamMap().items():
    print(f"  {param.name}: {value}")

# Making predictions on a new dataset
# Assuming test_set is your test DataFrame
predictions = loaded_model.transform(test_set_with_features)

Loaded model parameters:
  featuresCol: features
  labelCol: category_label
  modelType: multinomial
  predictionCol: prediction
  probabilityCol: probability
  rawPredictionCol: rawPrediction
  smoothing: 1.0


In [125]:
predictions.select('description_filtered','prediction').show()

24/06/04 20:45:14 WARN DAGScheduler: Broadcasting large task binary with size 34.1 MiB


+--------------------+----------+
|description_filtered|prediction|
+--------------------+----------+
|[apple, watch, ev...|      26.0|
|[arthveda, fund, ...|       0.0|
|[bank, india, hea...|       0.0|
|[bank, looking, a...|       0.0|
|[bank, may, accel...|       0.0|
|[better, liquidit...|       0.0|
|[bharti, axa, lif...|       0.0|
|[bill, seek, repl...|       0.0|
|[central, bank, a...|       0.0|
|[cleanup, exercis...|       0.0|
|[court, toss, 21,...|      27.0|
|[credit, growth, ...|       0.0|
|[credit, rating, ...|       0.0|
|[deal, ergos, sta...|       0.0|
|[describes, brexi...|       2.0|
|[draftkings, repo...|       0.0|
|[farmer, longer, ...|       0.0|
|[financial, intel...|       0.0|
|[firstquarter, ad...|       0.0|
|[given, poor, fis...|       0.0|
+--------------------+----------+
only showing top 20 rows



In [269]:
#df.unpersist()

In [ ]:
#sc.stop()

In [52]:
#!pip install scikit-learn

In [103]:
import numpy as np
# Select the required column
df_features_labels = df.select("description_filtered", "category_label")

# Convert to Pandas DataFrame for compatibility with scikit-learn
pandas_df = df_features_labels.toPandas()


KeyboardInterrupt



In [13]:
pandas_df

,description_filtered,category_label
0,grocer recall prepared food onion tied salmone...,0.0
1,fitch affirms sabic outlook stable,0.0
2,global pert market growth prospect insight ana...,0.0
3,global tung oil market strategic assessment re...,0.0
4,fxcm pro partner devexperts provide liquidity ...,0.0
...,...,...
1716603,selena gomez development new tv show described...,31.0
1716604,selena gomez developing new television set dis...,31.0
1716605,hurricane irma hour angle already hit million ...,31.0
1716606,googleorg invests 1 million serve create latin...,31.0


In [17]:
u=pandas_df['description_filtered'].tolist()
u=np.array(u)

MemoryError: Unable to allocate 40.0 GiB for an array with shape (1716608,) and data type <U6248

In [12]:
# Extract features (X) and labels (y)
X = np.array(pandas_df['description_filtered'].tolist())
y = pandas_df['category_label'].values

# Collect features and labels
#features_labels = df_transformed.select("features", "category_label").collect()

MemoryError: Unable to allocate 40.0 GiB for an array with shape (1716608,) and data type <U6248

In [5]:
from pyspark.ml.feature import CountVectorizer, IDF, Tokenizer,HashingTF
from pyspark.ml import Pipeline

# Tokenizer to split the text into words
tokenizer = Tokenizer(inputCol="description_filtered", outputCol="words")

# CountVectorizer to convert text into a matrix of token counts
#vectorizer = CountVectorizer(inputCol="words", outputCol="rawFeatures")

num_features = 2**18

# HashingTF to convert text into a matrix of token counts
hashing_tf = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=num_features)

# IDF to transform the count matrix to a normalized tf-idf representation
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Pipeline to execute the steps
pipeline = Pipeline(stages=[tokenizer, hashing_tf, idf])

# Fit the pipeline to the DataFrame
model = pipeline.fit(df)

# Transform the DataFrame to get the features
df_transformed = model.transform(df)


In [6]:
df_transformed.show()

24/05/27 22:22:35 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


+--------------+--------------------+--------------------+--------------------+--------------------+
|category_label|description_filtered|               words|         rawFeatures|            features|
+--------------+--------------------+--------------------+--------------------+--------------------+
|           0.0|grocer recall pre...|[grocer, recall, ...|(262144,[7194,435...|(262144,[7194,435...|
|           0.0|fitch affirms sab...|[fitch, affirms, ...|(262144,[8578,250...|(262144,[8578,250...|
|           0.0|global pert marke...|[global, pert, ma...|(262144,[3968,814...|(262144,[3968,814...|
|           0.0|global tung oil m...|[global, tung, oi...|(262144,[8145,166...|(262144,[8145,166...|
|           0.0|fxcm pro partner ...|[fxcm, pro, partn...|(262144,[43204,63...|(262144,[43204,63...|
|           0.0|water bottle mark...|[water, bottle, m...|(262144,[8145,181...|(262144,[8145,181...|
|           0.0|solarwinds ceo ke...|[solarwinds, ceo,...|(262144,[69703,82...|(262144,[697

In [18]:
import numpy as np
# Select the required columns
df_features_labels = df_transformed.select("features", "category_label")

# Convert to Pandas DataFrame for compatibility with scikit-learn
pandas_df = df_features_labels.toPandas()



# Collect features and labels
#features_labels = df_transformed.select("features", "category_label").collect()

24/05/27 22:40:20 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7001c5322c00>>
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 

KeyboardInterrupt



In [20]:
# Extract features (X) and labels (y)
X = pandas_df['description_filtered'].tolist()
y = pandas_df['category_label'].values

In [23]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [27]:
# Initialize HashingVectorizer
num_features = 1000  # Number of features (adjust as needed)
hashing_vectorizer = HashingVectorizer(n_features=num_features)

# Transform the text data to a matrix of token counts
X_hashed = hashing_vectorizer.transform(X)

# Initialize TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Transform the token counts to TF-IDF representation
X_tfidf = tfidf_transformer.fit_transform(X_hashed)

# Convert the sparse matrix to a dense matrix if needed
#X_tfidf_dense = X_tfidf.toarray()

# Print the transformed features and labels
print("Features (X):", X_tfidf)
print("Labels (y):", y)

Features (X):   (0, 119)	0.37782184895348164
  (0, 223)	0.3391235897269139
  (0, 414)	0.289078028690946
  (0, 586)	-0.3586289023237469
  (0, 746)	0.3455405618475319
  (0, 821)	0.3645115364307206
  (0, 850)	-0.36019888904328323
  (0, 952)	0.3847789824682703
  (1, 22)	-0.5120927452407951
  (1, 497)	-0.43333623286050665
  (1, 526)	-0.4385721263195585
  (1, 609)	0.4190474719352858
  (1, 840)	0.42665493767469526
  (2, 65)	-0.26713891555915836
  (2, 96)	0.2918124569517634
  (2, 157)	0.26948567442972904
  (2, 158)	0.2105510043769622
  (2, 327)	0.22911812683825664
  (2, 365)	-0.24891786971452565
  (2, 414)	0.21919526806619602
  (2, 492)	0.24675371427892087
  (2, 502)	0.248481858012751
  (2, 522)	0.24726822194757517
  (2, 552)	0.20407683997015152
  (2, 679)	0.25954132470356356
  :	:
  (1716604, 903)	0.18057904782117315
  (1716604, 930)	-0.2502975234964976
  (1716605, 468)	0.4250842709017103
  (1716605, 493)	0.38489466592947336
  (1716605, 508)	-0.37879752260362043
  (1716605, 660)	0.37814468093

In [36]:
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.svm import SVC

# Define the evaluation function
def evaluate_learners(models, X, y):
    train_scores = []
    test_scores = []
    for model in models:
        cv_results = cross_validate(model, X, y, cv=5, return_train_score=True)
        train_scores.append(cv_results['train_score'])
        test_scores.append(cv_results['test_score'])
    return train_scores, test_scores

# Load a sample dataset, e.g., Iris dataset
#iris = datasets.load_iris()
#X, y = iris.data, iris.target

# Define the parameter grid for SVC
param_grid = {
    'C': [0.1, 1, 10, 100]
}

# Create a base SVC model
svc = SVC(kernel='linear')
svc

SVC(kernel='linear')

In [37]:
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search

GridSearchCV(cv=5, estimator=SVC(kernel='linear'), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100]}, scoring='accuracy')

In [ ]:
# Fit the grid search to the data
grid_search.fit(X_tfidf, y)

24/05/28 07:06:34 WARN TransportChannelHandler: Exception in connection from /10.152.184.160:46169
java.io.IOException: Connection timed out
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:223)
	at sun.nio.ch.IOUtil.read(IOUtil.java:192)
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:379)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:254)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:357)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:788)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeysOptimized(NioEventLoop.java:724)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeys(NioEventLoop.j

In [ ]:
# Get the best model
best_svc_model = grid_search.best_estimator_
models = [best_svc_model]

In [ ]:
# Evaluate the model
train_scores, test_scores = evaluate_learners(models, X, y)

# Print the results
print("Best model parameters:", grid_search.best_params_)
print("Training scores:", train_scores)
print("Test scores:", test_scores)